In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

### Load the data

In [2]:
train = pd.read_csv('input/train.csv').fillna(' ')
test = pd.read_csv('input/test.csv').fillna(' ')
class_names = ['toxic','severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
id              153164 non-null object
comment_text    153164 non-null object
dtypes: object(2)
memory usage: 2.3+ MB


In [5]:
train.head(100)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [6]:
train_text = train['comment_text']
test_text = train['comment_text']
all_text = pd.concat([train_text, test_text])

In [15]:
word_vectorizer = TfidfVectorizer(sublinear_tf=True,\
                                 strip_accents='unicode',\
                                 analyzer='word',\
                                 token_pattern=r'w{1,}',\
                                 stop_words='english',\
                                 ngram_range=(1,1),\
                                 max_features=1000)

In [16]:
word_vectorizer.fit(all_text)
train_word_feature = word_vectorizer.transform(train_text)
test_word_feature = word_vectorizer.transform(test_text)

In [17]:
char_vectorizer = TfidfVectorizer(sublinear_tf=True,\
                                 strip_accents='unicode',\
                                 analyzer='char',\
                                 stop_words='english',\
                                 ngram_range=(2,6),\
                                 max_features=5000)

In [18]:
char_vectorizer.fit(all_text)
train_char_feature = char_vectorizer.transform(train_text)
test_char_feature = char_vectorizer.transform(test_text)


MemoryError: 

In [11]:
scores = []

In [14]:
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_word_feature, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_word_feature, train_target)
    submission[class_name] = classifier.predict_proba(test_word_feature)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.5601292270829816


ValueError: Length of values does not match length of index